### Requied Libraries

In [1]:
import numpy as np
import pandas as pd

### Node of Decision Tree

In [2]:
class Node:
    def __init__(self, feature_index, threshold, left, right, info_gain, value):
        """
        Constructor for the Node class.

        Parameters:
        - feature_index (int): Index of the feature used for splitting at this node.
        - threshold (float): Threshold value for splitting at this node.
        - left (Node): Reference to the left child node.
        - right (Node): Reference to the right child node.
        - info_gain (float): Information gain from the split at this node.
        - value (varied): The class label if this is a leaf node.
        """
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

### Decision Tree Implementation

In [3]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split, max_depth):
       ''' 
        Constructor for the DecisionTreeClassifier class.
        
        Parameters:
        min_samples_split (int): The minimum number of samples required to split a node.
        max_depth (int): The maximum depth of the tree.
        '''
       self.min_samples_split = min_samples_split
       self.max_depth = max_depth
        
        
    def build_tree(self, dataset, curr_depth=0):
        ''' 
        Recursively builds the decision tree from the dataset.
        
        Parameters:
        dataset (array): The data used to build the tree.
        curr_depth (int): The current depth of the tree.
        
        Returns:
        dict: The root node of the built tree.
        '''
        
        # 1: Check stopping conditions
        if len(dataset) < self.min_samples_split or curr_depth == self.max_depth:
            # Create a leaf node and calculate the leaf value
            leaf_value = self.calculate_leaf_value(dataset[:, -1])
            return {'leaf': True, 'value': leaf_value}
        
        # 2: Find the best split
        best_split = self.get_best_split(dataset, len(dataset), len(dataset[0]) - 1)
        
        # 3: Check if information gain is positive
        if best_split['information_gain'] <= 0:
            # Create a leaf node and calculate the leaf value
            leaf_value = self.calculate_leaf_value(dataset[:, -1])
            return {'leaf': True, 'value': leaf_value}
        
        # 4: Recur on the left child
        left_child = self.build_tree(best_split['left_child'], curr_depth + 1)
        
        # 5: Recur on the right child
        right_child = self.build_tree(best_split['right_child'], curr_depth + 1)
        
        # 6: Create a decision node
        decision_node = {
            'leaf': False,
            'feature_index': best_split['feature_index'],
            'threshold': best_split['threshold'],
            'left_child': left_child,
            'right_child': right_child
        }
        
        # 7: Return the decision node
        return decision_node
        

    def get_best_split(self, dataset, num_samples, num_features):
        '''
        Finds the best split for the dataset.
        
        Parameters:
        dataset (array): The dataset to split.
        num_samples (int): Number of samples in the dataset.
        num_features (int): Number of features in the dataset.
        
        Returns:
        dict: Information about the best split.    
        '''

        # 1: Define dictionary to store the best split 
        best_split = {}

        # 2: loop over all the features
        for feature_index in range(num_features):
            feature_values = np.unique(dataset[:, feature_index])

        # 3: loop over all the feature values present in the data
            for threshold in feature_values:

        # 4: get current split
                left_child, right_child = self.split(dataset, feature_index, threshold)

        # 5: check if childs are not null
                if len(left_child) > 0 and len(right_child) > 0:

        # 6: compute information gain
                    mode = "entropy"
                    information_gain = self.information_gain(dataset, left_child, right_child, mode)

        # 7: update the best split if needed
                    if "information_gain" not in best_split or information_gain > best_split["information_gain"]:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "left_child": left_child,
                            "right_child": right_child,
                            "information_gain": information_gain,
                        }

        # 8: return best split
        return best_split
        

    def split(self, dataset, feature_index, threshold):
        """
        Splits the dataset based on the given feature index and threshold.

        Parameters:
        - dataset (array): The dataset to split.
        - feature_index (int): The index of the feature used for splitting.
        - threshold (float): The threshold value for splitting.

        Returns:
        - tuple: Two subsets of the dataset split by the threshold.
        """
        left_subset = dataset[dataset[:, feature_index] <= threshold]
        right_subset = dataset[dataset[:, feature_index] > threshold]
        return left_subset, right_subset     


    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        """
        Computes the information gain of a split.

        Parameters:
        - parent (array): The parent dataset before the split.
        - l_child (array): The left child dataset after the split.
        - r_child (array): The right child dataset after the split.
        - mode (str): The criterion for information gain ('gini' or 'entropy').

        Returns:
        - float: The information gain of the split.
        """
        if mode == "gini":
            parent_impurity = self.gini_index(parent)
            l_child_impurity = self.gini_index(l_child)
            r_child_impurity = self.gini_index(r_child)
        else:
            parent_impurity = self.entropy(parent)
            l_child_impurity = self.entropy(l_child)
            r_child_impurity = self.entropy(r_child)

        parent_size = len(parent)
        l_child_size = len(l_child)
        r_child_size = len(r_child)
        total_child_size = l_child_size + r_child_size

        information_gain = parent_impurity - (
            (l_child_size / total_child_size) * l_child_impurity
            + (r_child_size / total_child_size) * r_child_impurity
        )

        return information_gain
    

    def entropy(self, y):
        """
        Computes the entropy of a dataset.

        Parameters:
        - y (array): The labels of the dataset.

        Returns:
        - float: The entropy of the dataset.
        """
        class_counts = np.unique(y, return_counts=True)[1]
        class_probabilities = class_counts / len(y)
        entropy = -np.sum(class_probabilities * np.log2(class_probabilities))
        return entropy
    

    def gini_index(self, y):
        """
        Computes the Gini index of a dataset.

        Parameters:
        - y (array): The labels of the dataset.

        Returns:
        - float: The Gini index of the dataset.
        """
        class_counts = np.unique(y, return_counts=True)[1]
        class_probabilities = class_counts / len(y)
        gini_index = 1 - np.sum(class_probabilities ** 2)
        return gini_index


    def calculate_leaf_value(self, Y):
        """
        Determines the value of a leaf node (most common class label).

        Parameters:
        - Y (array): The labels of the data in the leaf node.

        Returns:
        - varied: The most common class label in the leaf node.
        """
        class_counts = np.unique(Y, return_counts=True)
        most_common_label = class_counts[0][np.argmax(class_counts[1])]
        return most_common_label
    

    def print_tree(self, tree, indent="   "):
        ''' 
        Prints the tree in a readable format.
        
        Parameters:
        tree (Node, optional): The current node to print. If None, print from the root.
        indent (str): The indentation string to format the tree structure.
        '''
        if tree is None:
            return

        if isinstance(tree, dict):
            if tree['leaf']:
                print(indent + "Leaf Node: ", tree['value'])
            else:
                print(indent + "Feature", tree['feature_index'], "<=", tree['threshold'], "?")

                print(indent + "--> True:")
                self.print_tree(tree['left_child'], indent + "  ")

                print(indent + "--> False:")
                self.print_tree(tree['right_child'], indent + "  ")
    

    def fit(self, X, Y):
        ''' 
        Trains the decision tree classifier.
        
        Parameters:
        X (array): The features of the training data.
        Y (array): The labels of the training data.
        '''
        dataset = np.column_stack((X, Y))
        self.tree = self.build_tree(dataset)
    

    def predict(self, X):
        """
        Predicts the class labels for the given data.

        Parameters:
        - X (array-like): The input data.

        Returns:
        - array: The predicted class labels.
        """
        predictions = []

        for sample in X:
            node = self.tree

            while not node['leaf']:
                if sample[node['feature_index']] <= node['threshold']:
                    node = node['left_child']
                else:
                    node = node['right_child']

            predictions.append(node['value'])

        return np.array(predictions)
    
    
    def make_prediction(self, x, tree):
        ''' 
        Predicts a class label for a single data point.
        
        Parameters:
        x (array): The features of the single data point.
        tree (Node): The current node in the tree during the recursive prediction.
        
        Returns:
        varied: The predicted class label.
        '''
        if tree['leaf']:
            return tree['value']
        
        if x[tree['feature_index']] <= tree['threshold']:
            return self.make_prediction(x, tree['left_child'])
        else:
            return self.make_prediction(x, tree['right_child'])

## Read Dataset

In [4]:
train_dataset = pd.read_csv('Dataset/train.csv')
test_dataset = pd.read_csv('Dataset/test.csv')

## Pre-Process

Drop Id columns but keep y_test column Ids

In [5]:
y_test_id = test_dataset['Id']

# Drop the 'Id' column
train_dataset = train_dataset.drop('Id', axis=1)
test_dataset = test_dataset.drop('Id', axis=1)

Encode Species

In [6]:
conversion_dict = {
    "Iris-setosa": 0,
    "Iris-versicolor": 1,
    "Iris-virginica": 2
}

train_dataset["Species"] = train_dataset["Species"].map(conversion_dict)
test_dataset["Species"] = test_dataset["Species"].map(conversion_dict)

## Split Xs and Ys

In [7]:
target_column = 'Species'

X_train = train_dataset.drop(target_column, axis=1)
y_train = train_dataset[target_column]

X_test = test_dataset.drop(target_column, axis=1)
y_test = test_dataset[target_column]

In [8]:
X_train.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
0,5.4,3.0,4.5,1.5
1,7.7,2.8,6.7,2.0
2,5.2,3.4,1.4,0.2
3,4.8,3.4,1.9,0.2
4,6.6,3.0,4.4,1.4


In [9]:
print(y_train.to_list())

[1, 2, 0, 0, 1, 2, 1, 1, 1, 2, 2, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 1, 2, 0, 0, 0, 1, 2, 2, 1, 0, 1, 1, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 2, 0, 2, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 2, 2, 1, 1, 2, 0, 0, 2, 1, 0, 2, 2, 0, 0, 1, 2, 1, 2, 0, 1, 0, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 0, 0]


## Accuracy Score From Scratch

In [10]:
def acc(y_true, y_pred):
    correct = 0
    total = 0
    for true_label, pred_label in zip(y_true, y_pred):
        if true_label == pred_label:
            correct += 1
        total += 1
    accuracy = correct / total
    return accuracy

# Decision Tree

Create Instance of DecisionTreeClassifier Class

In [11]:
tree = DecisionTreeClassifier(4, 10)

tree.fit(X_train, y_train)

predictions = tree.predict(X_test.values)
accuracy = acc(y_test, predictions)
print(accuracy)

0.9
